In [5]:
# Creat demo.xlsx

In [6]:
# import numpy as np
# import pandas as pd
# np.random.seed(1)
# data = pd.DataFrame({
#     'label': np.random.randint(0, 2, 500),
#     'age': np.random.randint(18, 70, 500),
#     'sex': np.random.randint(0, 2, 500),
#     'height': np.random.randint(140, 200, 500),
#     'weight': np.random.randint(40, 100, 500)
# })
# data.to_excel('demo.xlsx', index=False)

In [7]:
# Fast Evaluation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

datafile = 'demo.xlsx'
data = pd.read_excel(datafile)
X, y = data.values[:, 1:], data.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
result = accuracy_score(y_test, y_predict)
print(result)